In [ ]:
import torch
from torch.utils.data import DataLoader

from utils.data_reader import get_train_dev_test_data, get_review_dict
from utils.data_set import NarreDataset
from utils.log_hepler import logger
from utils.train_helper import load_model

train_data, dev_data, test_data = get_train_dev_test_data()
model = load_model("model/checkpoints/NarreModel_20230424112221.pt")
model.config.device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(model.config.device)
loss = torch.nn.MSELoss()

review_by_user, review_by_item = get_review_dict("test")

print (type (review_by_user))
print (review_by_user)
dataset = NarreDataset(test_data, review_by_user, review_by_item, model.config)
data_iter = DataLoader(dataset, batch_size=model.config.batch_size, shuffle=True)

: 

In [ ]:
def eval_model(model, data_iter, loss):
    model.eval()
    model_name = model.__class__.__name__
    config: BaseConfig = model.config
    logger.debug("Evaluating %s..." % model_name)

    with torch.no_grad():
        predicts = []
        ratings = []
        for batch_id, iter_i in enumerate(data_iter):
            user_review, user_id, item_id_per_review, item_review, item_id, user_id_per_review, rating = iter_i

            print ("User review : ", type (user_review), user_review)
            print ("User ID : ", type (user_id), user_id)
            print ("Item ID per review : ", type (item_id_per_review), item_id_per_review)
            print ("Item review : ", type (item_review), item_review)
            print ("Item ID : ", type (item_id), item_id)
            print ("USer ID per review : ", type (user_id_per_review), user_id_per_review)
            print ("Rating : ", type (rating), rating)
            
            user_review = user_review.to(config.device)
            user_id = user_id.to(config.device)
            item_id_per_review = item_id_per_review.to(config.device)

            item_review = item_review.to(config.device)
            item_id = item_id.to(config.device)
            user_id_per_review = user_id_per_review.to(config.device)

            rating = rating.to(config.device)
            
            print ("User review : ", type (user_review), user_review)
            print ("User ID : ", type (user_id), user_id)
            print ("Item ID per review : ", type (item_id_per_review), item_id_per_review)
            print ("Item review : ", type (item_review), item_review)
            print ("Item ID : ", type (item_id), item_id)
            print ("USer ID per review : ", type (user_id_per_review), user_id_per_review)
            print ("Rating : ", type (rating), rating)
            
            break

            predict = model(user_review, user_id, item_id_per_review, item_review, item_id, user_id_per_review)
            predicts.append(predict)
            ratings.append(rating)

        predicts = torch.cat(predicts)
        ratings = torch.cat(ratings)
        return loss(predicts, ratings).item()

In [ ]:
logger.info(f"Loss on test dataset: {eval_model(model, data_iter, loss)}")